
Example of using RLlib's debug callbacks.<br>
Here we use callbacks to track the average CartPole pole angle magnitude as a<br>
custom metric.<br>


In [1]:
from typing import Dict, Tuple
import argparse
import numpy as np
import os

In [2]:
import ray
from ray import air, tune
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from ray.rllib.env import BaseEnv
from ray.rllib.evaluation import Episode, RolloutWorker
from ray.rllib.policy import Policy
from ray.rllib.policy.sample_batch import SampleBatch

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument(
    "--framework",
    choices=["tf", "tf2", "torch"],
    default="torch",
    help="The DL framework specifier.",
)
parser.add_argument("--stop-iters", type=int, default=2000)

_StoreAction(option_strings=['--stop-iters'], dest='stop_iters', nargs=None, const=None, default=2000, type=<class 'int'>, choices=None, required=False, help=None, metavar=None)

In [4]:
class MyCallbacks(DefaultCallbacks):
    def on_episode_start(
        self,
        *,
        worker: RolloutWorker,
        base_env: BaseEnv,
        policies: Dict[str, Policy],
        episode: Episode,
        env_index: int,
        **kwargs
    ):
        # Make sure this episode has just been started (only initial obs
        # logged so far).
        assert episode.length == 0, (
            "ERROR: `on_episode_start()` callback should be called right "
            "after env reset!"
        )
        print("episode {} (env-idx={}) started.".format(episode.episode_id, env_index))
        episode.user_data["distance"] = []
        episode.hist_data["distance"] = []
    def on_episode_step(
        self,
        *,
        worker: RolloutWorker,
        base_env: BaseEnv,
        policies: Dict[str, Policy],
        episode: Episode,
        env_index: int,
        **kwargs
    ):
        # Make sure this episode is ongoing.
        assert episode.length > 0, (
            "ERROR: `on_episode_step()` callback should not be called right "
            "after env reset!"
        )
        pole_angle = abs(episode.last_observation_for()[2])
        raw_angle = abs(episode.last_raw_obs_for()[2])

        episode.user_data["distance"].append(pole_angle)
    def on_episode_end(
        self,
        *,
        worker: RolloutWorker,
        base_env: BaseEnv,
        policies: Dict[str, Policy],
        episode: Episode,
        env_index: int,
        **kwargs
    ):
        # Check if there are multiple episodes in a batch, i.e.
        # "batch_mode": "truncate_episodes".
        if worker.policy_config["batch_mode"] == "truncate_episodes":
            # Make sure this episode is really done.
            assert episode.batch_builder.policy_collectors["default_policy"].batches[
                -1
            ]["dones"][-1], (
                "ERROR: `on_episode_end()` should only be called "
                "after episode is done!"
            )
        pole_angle = np.mean(episode.user_data["distance"])
        print(
            "episode {} (env-idx={}) ended with length {} and pole "
            "angles {}".format(
                episode.episode_id, env_index, episode.length, pole_angle
            )
        )
        episode.custom_metrics["distance"] = pole_angle
        episode.hist_data["distance"] = episode.user_data["distance"]
    def on_sample_end(self, *, worker: RolloutWorker, samples: SampleBatch, **kwargs):
        print("returned sample batch of size {}".format(samples.count))
    def on_train_result(self, *, algorithm, result: dict, **kwargs):
        print(
            "Algorithm.train() result: {} -> {} episodes".format(
                algorithm, result["episodes_this_iter"]
            )
        )
        # you can mutate the result dict to add new fields to return
        result["callback_ok"] = True
    def on_learn_on_batch(
        self, *, policy: Policy, train_batch: SampleBatch, result: dict, **kwargs
    ) -> None:
        result["sum_actions_in_train_batch"] = train_batch["actions"].sum()
        print(
            "policy.learn_on_batch() result: {} -> sum actions: {}".format(
                policy, result["sum_actions_in_train_batch"]
            )
        )
    def on_postprocess_trajectory(
        self,
        *,
        worker: RolloutWorker,
        episode: Episode,
        agent_id: str,
        policy_id: str,
        policies: Dict[str, Policy],
        postprocessed_batch: SampleBatch,
        original_batches: Dict[str, Tuple[Policy, SampleBatch]],
        **kwargs
    ):
        print("postprocessed {} steps".format(postprocessed_batch.count))
        if "num_batches" not in episode.custom_metrics:
            episode.custom_metrics["num_batches"] = 0
        episode.custom_metrics["num_batches"] += 1

In [5]:
from cassie import CassieEnv
import constants as c 
import functions as f 
import numpy as np
import loader as l 
import caps as cp 
import torch
from ray.tune.registry import register_env
from ray.rllib.agents.ppo import PPOTrainer
import mediapy as media 
from loader import Loader

In [6]:
loader = Loader(logdir = "./custom_metrics/logs/", simdir = "./custom_metrics/sims/")

In [7]:
config = loader.load_config("../proto.yaml")

{'training': {'gamma': 0.99, 'lr': 0.01, 'train_batch_size': 1000, 'model': {'fcnet_hiddens': [64, 64], 'fcnet_activation': 'swish', 'vf_share_layers': False, 'free_log_std': True}, 'optimizer': {'type': 'adam', 'eps': '1e-06'}, 'use_critic': True, 'use_gae': True, 'lambda_': 0.95, 'kl_coeff': 0.2, 'sgd_minibatch_size': 200, 'num_sgd_iter': 10, 'vf_loss_coeff': 0.5, 'entropy_coeff': 0.01, 'clip_param': 0.2, 'kl_target': 0.01}, 'environment': {'env': 'cassie-v0', 'disable_env_checking': True}, 'framework': {'framework': 'torch'}, 'rollouts': {'recreate_failed_workers': True, 'num_workers': 1, 'num_envs_per_worker': 1, 'rollout_fragment_length': 'auto', 'batch_mode': 'truncate_episodes', 'observation_filter': 'NoFilter', 'reuse_actors': True}, 'evaluation': {'evaluation_interval': 2, 'evaluation_duration': 10}}


In [8]:
splitted = loader.split_config(config)

In [9]:
combined = {**splitted.get("environment",{}), **splitted.get("rollouts",{}), **splitted.get("checkpointing",{}), **splitted.get("debugging",{}), **splitted.get("training",{}), **splitted.get("framework",{}), **splitted.get("resources",{}), **splitted.get("evaluation",{})}
combined["callbacks"] = MyCallbacks

In [10]:
register_env("cassie-v0", lambda config: CassieEnv(config))

In [18]:
trainer = PPOTrainer(config=combined, env="CartPole-v1")
# trainer.callbacks = [MyCallbacks]

2023-04-23 09:51:55,948	WARNING deprecation.py:50 -- DeprecationWarning: `algo = Algorithm(env='CartPole-v1', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('CartPole-v1').build()` instead. This will raise an error in the future!
2023-04-23 09:51:58,797	WARNING util.py:67 -- Install gputil for GPU system monitoring.


In [19]:
result = trainer.train()

(RolloutWorker pid=8196) 2023-04-23 09:52:00,837	ERROR actor_manager.py:177 -- Worker exception, recreating: ERROR: `on_episode_start()` callback should be called right after env reset!
(RolloutWorker pid=8196) Traceback (most recent call last):
(RolloutWorker pid=8196)   File "c:\Users\Ajvendetta\AppData\Local\Programs\Python\Python310\lib\site-packages\ray\rllib\utils\actor_manager.py", line 174, in apply
(RolloutWorker pid=8196)     return func(self, *args, **kwargs)
(RolloutWorker pid=8196)   File "c:\Users\Ajvendetta\AppData\Local\Programs\Python\Python310\lib\site-packages\ray\rllib\execution\rollout_ops.py", line 86, in <lambda>
(RolloutWorker pid=8196)     lambda w: w.sample(), local_worker=False, healthy_only=True
(RolloutWorker pid=8196)   File "c:\Users\Ajvendetta\AppData\Local\Programs\Python\Python310\lib\site-packages\ray\util\tracing\tracing_helper.py", line 466, in _resume_span
(RolloutWorker pid=8196)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=8196) 

KeyError: 'actions'

In [11]:
if __name__ == "__main__":

    ray.init(ignore_reinit_error=True)
    tuner = tune.Tuner(
        "PG",
        run_config=air.RunConfig(
            stop={
                "training_iteration": 10,
            },
        ),
        param_space={
            "env": "CartPole-v1",
            "num_envs_per_worker": 2,
            "callbacks": MyCallbacks,
            "framework": "torch",
            # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
            "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
            # TODO(avnishn): This example uses functions specific to episode v1
            # that is not compatible with episode v2. Needs to be updated
            "enable_connectors": False,
        },
    )
    # there is only one trial involved.
    result = tuner.fit().get_best_result()

    # Verify episode-related custom metrics are there.
    custom_metrics = result.metrics["custom_metrics"]
    print(custom_metrics)
    assert "pole_angle_mean" in custom_metrics
    assert "pole_angle_min" in custom_metrics
    assert "pole_angle_max" in custom_metrics
    assert "num_batches_mean" in custom_metrics
    assert "callback_ok" in result.metrics

2023-04-23 08:27:06,717	INFO worker.py:1382 -- Calling ray.init() again after it has already been called.


(PG pid=22772) 2023-04-23 08:27:12,582	WARNING algorithm_config.py:596 -- Cannot create PGConfig from given `config_dict`! Property __stdout_file__ not supported.


(PG pid=22772) episode 243690006713247071 (env-idx=0) started.
(PG pid=22772) episode 474701696523161799 (env-idx=1) started.
(PG pid=22772) postprocessed 14 steps
(PG pid=22772) episode 474701696523161799 (env-idx=1) ended with length 14 and pole angles 0.11169251054525375
(PG pid=22772) episode 900463818452346912 (env-idx=1) started.
(PG pid=22772) postprocessed 23 steps
(PG pid=22772) episode 243690006713247071 (env-idx=0) ended with length 23 and pole angles 0.10716638714075089
(PG pid=22772) episode 931551982301474348 (env-idx=0) started.
(PG pid=22772) postprocessed 29 steps
(PG pid=22772) episode 900463818452346912 (env-idx=1) ended with length 29 and pole angles 0.06946299970149994
(PG pid=22772) episode 46412943169667800 (env-idx=1) started.
(PG pid=22772) postprocessed 24 steps
(PG pid=22772) episode 931551982301474348 (env-idx=0) ended with length 24 and pole angles 0.08858674764633179
(PG pid=22772) episode 773014677554739165 (env-idx=0) started.
(PG pid=22772) postprocesse

Trial name,agent_timesteps_total,callback_ok,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PG_CartPole-v1_de7e6_00000,2000,True,{},"{'num_env_steps_sampled': 2000, 'num_env_steps_trained': 2000, 'num_agent_steps_sampled': 2000, 'num_agent_steps_trained': 2000}","{'num_batches_mean': 1.2297297297297298, 'num_batches_min': 1, 'num_batches_max': 2, 'pole_angle_mean': 0.08164537, 'pole_angle_min': 0.03770415, 'pole_angle_max': 0.124951355}",2023-04-23_08-27-14,True,26.5946,{},78,26.5946,10,6,74,21b3001429664728806650d7e6654cf7,DESKTOP-U8A332F,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'policy_loss': 10.180950164794922, 'cur_lr': 0.0004}, 'model': {}, 'custom_metrics': {'sum_actions_in_train_batch': 98.0}, 'num_agent_steps_trained': 200.0, 'num_grad_updates_lifetime': 10.0, 'diff_num_grad_updates_vs_sampler_policy': 0.0}}, 'num_env_steps_sampled': 2000, 'num_env_steps_trained': 2000, 'num_agent_steps_sampled': 2000, 'num_agent_steps_trained': 2000}",10,127.0.0.1,2000,2000,2000,200,2000,200,0,0,0,0,200,{},22772,{},{},{},"{'mean_raw_obs_processing_ms': 0.28717190123902475, 'mean_inference_ms': 0.7847068388578435, 'mean_action_processing_ms': 0.11854633483138652, 'mean_env_wait_ms': 0.061531060757738516, 'mean_env_render_ms': 0.0}",1.24064,0.145749,1.24064,"{'training_iteration_time_ms': 123.265, 'load_time_ms': 0.3, 'load_throughput': 666291.342, 'learn_time_ms': 5.038, 'learn_throughput': 39700.554, 'synch_weights_time_ms': 0.0}",1682231234,0,2000,10,de7e6_00000,0.0624025


(PG pid=22772) postprocessed 12 steps
(PG pid=22772) episode 224906159115926065 (env-idx=1) ended with length 12 and pole angles 0.08085270971059799
(PG pid=22772) episode 546051817624544973 (env-idx=1) started.
(PG pid=22772) postprocessed 28 steps
(PG pid=22772) episode 849675155454027208 (env-idx=0) ended with length 28 and pole angles 0.06950537860393524
(PG pid=22772) episode 901692773908464672 (env-idx=0) started.
(PG pid=22772) postprocessed 11 steps
(PG pid=22772) episode 546051817624544973 (env-idx=1) ended with length 11 and pole angles 0.0999380424618721
(PG pid=22772) episode 493934757727956784 (env-idx=1) started.
(PG pid=22772) postprocessed 5 steps
(PG pid=22772) postprocessed 2 steps
(PG pid=22772) returned sample batch of size 200
(PG pid=22772) policy.learn_on_batch() result: PGTorchPolicy -> sum actions: 108
(PG pid=22772) Algorithm.train() result: PG -> 10 episodes
(PG pid=22772) postprocessed 6 steps
(PG pid=22772) episode 901692773908464672 (env-idx=0) ended with 

2023-04-23 08:27:15,335	INFO tune.py:798 -- Total run time: 8.61 seconds (8.36 seconds for the tuning loop).


{'num_batches_mean': 1.2297297297297298, 'num_batches_min': 1, 'num_batches_max': 2, 'pole_angle_mean': 0.08164537, 'pole_angle_min': 0.03770415, 'pole_angle_max': 0.124951355}
